In [1]:
import sys, os
sys.path.append('C:\deep-learning-from-scratch-master') 
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *


class DeepConvNet:
    """정확도 99% 이상의 고정밀 합성곱 신경망

    네트워크 구성은 아래와 같음
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    """
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 가중치 초기화===========
        # 각 층의 뉴런 하나당 앞 층의 몇 개 뉴런과 연결되는가（TODO: 자동 계산되게 바꿀 것）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLU를 사용할 때의 권장 초깃값
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # 계층 생성===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]

In [2]:
import sys, os
sys.path.append('C:\deep-learning-from-scratch-master')  
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()  
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=20, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr':0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보관
network.save_params("deep_convnet_params.pkl")
print("Saved Network Parameters!")

train loss:2.3010117661031013
=== epoch:1, train acc:0.115, test acc:0.108 ===
train loss:2.284569640420327
train loss:2.251622435418108
train loss:2.2510426652522817
train loss:2.3138054553770555
train loss:2.2587934372281238
train loss:2.2645213253882512
train loss:2.2452238509492513
train loss:2.247810114471489
train loss:2.2499466261083567
train loss:2.2139028981607005
train loss:2.2351346822720313
train loss:2.15285451275638
train loss:2.1957379808921496
train loss:2.155389781083886
train loss:2.1804601817769993
train loss:2.096009170024893
train loss:2.203151571489256
train loss:2.070936467269962
train loss:2.064361411313009
train loss:2.157145704605557
train loss:2.090334056140737
train loss:1.9989235680090365
train loss:2.075746469523381
train loss:1.9812640610246328
train loss:2.0115711980510924
train loss:2.0441021800021217
train loss:1.9989693211733033
train loss:2.0352503575605976
train loss:2.004960628139425
train loss:1.9903624093667154
train loss:1.844940353894261
train 

train loss:1.302763119773286
train loss:1.1508207596013698
train loss:1.345596556347781
train loss:1.0332631061281947
train loss:1.1373916855679191
train loss:1.239179528054233
train loss:1.228902501757224
train loss:1.2731505978253452
train loss:1.336982782390798
train loss:1.2335382947573796
train loss:1.2537839233563606
train loss:1.2846901932059474
train loss:1.124870651696935
train loss:1.162035801703898
train loss:1.0344230444593223
train loss:1.0072474768332982
train loss:1.197498031654225
train loss:1.2294674720934091
train loss:1.2294983882493513
train loss:0.9165247277061569
train loss:1.1928496101252513
train loss:1.160279112932101
train loss:1.1641690265172502
train loss:1.1205497850890365
train loss:1.2924082348950756
train loss:1.1696463868562115
train loss:1.183996059397118
train loss:1.3988128558829669
train loss:1.2603879102597495
train loss:1.2567595347101834
train loss:1.090022080436286
train loss:1.1593064552597747
train loss:1.021249404628274
train loss:1.246660903

train loss:1.1331122348256677
train loss:1.145999320511366
train loss:1.1276137336720968
train loss:0.9185114189672473
train loss:1.118778551483663
train loss:0.9621527558514886
train loss:1.0825808980697287
train loss:1.0522795224128902
train loss:1.2510963450205865
train loss:1.0418174023878553
train loss:0.8918750807629826
train loss:1.0974733939478831
train loss:1.0129822545930531
train loss:1.19453875503885
train loss:1.0431677328590312
train loss:1.0437646582030695
train loss:1.1514525704017564
train loss:1.0488787599456755
train loss:0.9717985312281529
train loss:0.9874094195945148
train loss:1.0399622291543154
train loss:1.0627871180666522
train loss:0.9345737462186338
train loss:0.9704859431679523
train loss:1.4008141725005376
train loss:1.0500497134597186
train loss:0.9679303596528274
train loss:1.002267369493403
train loss:1.0850501841631222
train loss:1.002597434502102
train loss:1.0657060592524648
train loss:1.040250151048982
train loss:1.134883587376773
train loss:1.22543

train loss:0.9520713974488065
train loss:0.937041311267818
train loss:0.8172362898460235
train loss:0.8945618698360875
train loss:1.1046689008055044
train loss:0.9450138682180961
train loss:1.0632091406553115
train loss:1.2554044622717773
train loss:1.099995109823551
train loss:0.9733349764679946
train loss:0.9498472541664011
train loss:1.0559490488823513
train loss:1.013269142669368
train loss:1.007368916671979
train loss:0.944818933430104
train loss:0.9814430759165222
train loss:1.100231953559275
train loss:0.9781188215098071
train loss:0.9279812246410757
train loss:0.9948793554880179
train loss:0.9959741580616666
train loss:1.0206264736597639
train loss:0.9220512990006778
train loss:0.9654729168333018
train loss:1.0075169104472794
train loss:0.9368106594571547
train loss:0.9097952471747464
train loss:1.0778034810959796
train loss:1.17420857201914
train loss:0.8287791700349824
train loss:1.1122129244017205
train loss:0.965786505793317
train loss:0.9066670372849291
train loss:1.034668

train loss:0.8620922182019971
train loss:1.0210368297488754
train loss:0.9015093036031575
train loss:1.0444540900794466
train loss:1.1108275379106798
train loss:1.0264832352919107
train loss:0.9668694719804389
train loss:1.1084954393454323
train loss:0.9742931365365994
train loss:1.0595443697561766
train loss:1.0917720162757583
train loss:0.791135520813387
train loss:1.0528302498244277
train loss:0.9960065285323279
train loss:1.040386858520864
train loss:1.07986466647214
train loss:0.9782943253147713
train loss:1.0205893440162226
train loss:0.8385310625522958
train loss:0.9974809412497543
train loss:0.9601413145664863
train loss:1.0975886322369397
train loss:0.9947815217314195
train loss:0.901448684285649
train loss:1.0941287922480447
train loss:0.9710447261488838
train loss:0.8793805092823991
train loss:1.1746735492448308
train loss:1.0680495694497865
train loss:0.9126490851689831
train loss:0.9149413315846422
train loss:1.0788763772555798
train loss:0.9630341994260263
train loss:1.00

train loss:1.1636592906178256
train loss:1.0557131742612471
train loss:0.897740969273208
train loss:1.1042560449961316
train loss:0.9988717636295881
train loss:1.0976424855404323
train loss:1.0624117680374932
train loss:0.7731592588924164
train loss:1.0331769988337975
train loss:0.8924742302553876
train loss:1.0048896010531323
train loss:0.7448670788787187
train loss:0.9176506959016709
train loss:0.979446805839789
train loss:1.0043784560393536
train loss:0.9506399069122259
train loss:0.8767637454951348
train loss:0.9562617675586147
train loss:0.958662321915146
train loss:1.0870839013201126
train loss:0.8841717379417786
train loss:0.9873770418072186
train loss:1.0112605460988369
train loss:1.1028867468385077
train loss:0.8812306575778485
train loss:0.8599349943578696
train loss:0.9056731219027148
train loss:0.9822687549742024
train loss:0.8197331540709999
train loss:1.0953334305156657
train loss:1.0884314436539395
train loss:0.7787484918729612
train loss:1.115324751699149
train loss:0.8

train loss:0.858690437149722
train loss:0.8397460272777515
train loss:0.9878840866104757
train loss:1.0609746870017434
train loss:0.8567996049674715
train loss:1.0791633889795083
train loss:1.1115007002124326
train loss:1.0275924327169426
train loss:0.9673046895594176
train loss:0.8854682928870726
train loss:0.8111631892984558
train loss:0.756643534470103
train loss:0.9359937707922076
train loss:1.0717201323677488
train loss:1.0552362198374743
train loss:1.1582327342083767
train loss:0.9711902000078797
train loss:0.9586092202414457
train loss:0.9341625622244482
train loss:0.9631849643379183
train loss:1.159511794326627
train loss:1.0319990571817113
train loss:0.8922628615019563
train loss:0.9289069317517854
train loss:0.9529812068223983
train loss:1.0037405837406026
train loss:0.9856295075672031
train loss:0.8616498090539951
train loss:1.0655426679882474
train loss:0.9752000497355658
train loss:0.9543005818659489
train loss:1.0163928660718908
train loss:0.8769397584111387
train loss:1.

train loss:0.886261083990778
train loss:0.7547851205431616
train loss:1.0265710545319104
train loss:0.8477605083416598
train loss:0.793601438023676
train loss:1.1240570050583367
train loss:0.9014209012571008
train loss:0.9611696154072065
train loss:0.9128742137121527
train loss:1.0940664195354908
train loss:1.014326886053879
train loss:0.9169703520374594
train loss:0.9265463157703541
train loss:1.0221977660576176
train loss:0.8754092746066441
train loss:1.0038930363173102
train loss:0.7357155427192883
train loss:0.9623443434570512
train loss:0.8564551061855744
train loss:0.7765420243541005
train loss:0.8572356888257847
train loss:0.9676186336063176
train loss:1.0048108816217969
train loss:0.8236591861920626
train loss:0.9693151640468184
train loss:0.9091383128750222
train loss:0.9376631983916071
train loss:0.9850139282625721
train loss:0.9849474398714042
train loss:0.9330128510136894
train loss:0.9238832127170714
train loss:0.9632665826103206
train loss:0.9147057337031189
train loss:0.

KeyboardInterrupt: 